<div style="display: flex; align-items: center;">
  <h1 style="font-size: 4.5em; margin-right: 15px;">Physics Aware Recurrent Convolutional Neural Network (PARC): Navier-Stokes Demo</h1>
  <div>
    <img src="../img/VIL_logo.png" width="190" alt="Image 1" style="margin-right: 1px;" />
    <img src="../img/uva.png" width="190" alt="Image 2" style="margin-right: 1px;" />
    <img src="../img/iowa.png" width="190" alt="Image 3" />
  </div>
</div>
<p>A customizable framework to embed physics in Deep Learning. PARC's formulation is inspired by Advection-Diffusion-Reaction processes and uses an Inductive Bias approach to constrain the Neural Network.</p>




## Why PARC?
<p>PARC brings together deep learning with first prinicples physics - its recurrent convolutional core is able to learn complex spatiotemporal patterns, while built-in biases (conservation laws & advection-diffusion operators) ensure every prediction is physically plausible. PARC is constructed in such a manner that is does not need to "re-learn" fundemental dynamics - allowing faster training with far less data.</p>

<p>PINN (Physics-Informed Neural Networks) exists as an adjacent model to PARC. While PINN may preform decently on generics, it struggles with advection or chaotic/non-linear systems - net leading to higher computational cost. In such situations PARC is deemed the superior model with it being more scalable, efficient, and accurate than other physics-based models.</p>

### Goal
<p>The goal of this notebook is to walk you through the use cases of PARC via the Navier-Stokes equation. At the end of this notebook you will be presented with visualization in accordance to the equations representing conservation of momentum for fluids and fully describing fluid motion.</p>

The notebook will guide you through from start to finish in preparing, training, and modeling a physics-based equation. The notebook will cover the following:

- loading and preparinf data for Navier-Stokes
- Using PARCv2 Model to learn and predict time evolution of $u(x,t)$
- Evaluating the model's preformance and compare predicted resutls to ground truth

### Internal General PARC PDE
<p>Below is the general form of the partial differential equation that PARCv2 is learning - and its inital boundaries. In the case of the Navier-Stokes equations - we can describe the certain vairables to represent the nessearcy points in which PARC tries to model after the equation.</p>

$$
\frac{\partial X}{\partial t} = f(X,\mu) + \epsilon
$$

- $X$ is the fields of interest - Temperature, Pressure, Reynolds, Velocities (U & V)
- $μ$ is the microstructure


### Setting Up
<p>This document serves as a guide to training a PARC model for the Navier-Stokes equation. Here are the inital steps to take before you begin training your PARC model!<p>

Download & Prepare Data:
- Download the data from https://zenodo.org/records/13909869 and unzip it.
- Extact the data and ensure that it placed in the following directory: PARCtorch/PARCtorch/data
- If needed update the paths in the notebook accordlingly (for train_dir, test_dir)

Install PARCtorch:
- Ensure PARCtorch is installed in your Python Environemnt - view instructions on installation at: https://github.com/baeklab/PARCtorch
- Note: Ensure to be in the same environemnt/kernel when continuing in this notebook.

Happy Modeling!

## Download & Prepare Data
PARCtorch comes with a built-in dataset containing Navier-Stokes simulations. The following script will allow you to access the dataset.

In [ ]:
import torch
from PARCtorch.utils.common import CACHE_DIR

from PARCtorch.datasets.utils import download, extract_zip

import os
from pathlib import Path
import numpy as np

class Data:
    def __init__(self, data=None, traj_varying=False, time_varying=False, space_varying=False):
        self.traj_varying = traj_varying
        self.time_varying = time_varying
        self.space_varying = space_varying
        self.data = torch.as_tensor(data)
        # TODO: assert(len(data.shape) >= traj_varying + time_varying + space_varying)

    @classmethod
    def __torch_function__(self, func, types, args=(), kwargs=None):
        # Forward any torch function to self.data
        if kwargs is None:
            kwargs = {}
        args = tuple(a.data if isinstance(a, Data) else a for a in args)
        return func(*args, **kwargs)
    

    def __getitem__(self, index):
        return self.data[index]
    
    def __eq__(self, other):
        return self.data == other
    
    def __ne__(self, other):
        return self.data != other
    
    def __lt__(self, other):
        return self.data < other
    
    def __le__(self, other):
        return self.data <= other
    
    def __gt__(self, other):
        return self.data > other
    
    def __ge__(self, other):
        return self.data >= other
    
    def __len__(self):
        if self.data is None:
            return 0
        return len(self.data)
    
    def __repr__(self):
        return f"<PARC Data Class: {self.data}, traj_varying={self.traj_varying}, time_varying={self.time_varying}, space_varying={self.space_varying}>"
    
    @property
    def shape(self):
        return self.data.shape
    

    @property
    def rank(self):
        if self.data is None:
            return -1
        
        rank = len(self.data.shape) - self.traj_varying - self.time_varying
        if isinstance(self.space_varying, list):
            rank -= sum(self.space_varying)
        else:
            rank -= self.space_varying

        return rank

        
    @property
    def type(self):
        type_str = ''
        if self.traj_varying:
            type_str = 'Variable'
        else:
            type_str = 'Constant'
        
        if self.time_varying:
            type_str += ' time-dependent'
        else:
            type_str += ' time-independent'
    
        rank = self.rank
        if rank == 0:
            type_str += ' scalar'
        elif rank == 1:
            type_str += ' vector'
        else:
            type_str += f' rank-{rank} tensor'

        if any(self.space_varying):
            type_str += ' field'

        return type_str
            


class Dataset(torch.utils.data.Dataset):
    '''
    Data is 
    '''
    url = "https://zenodo.org/records/13909869/files/NavierStokes.zip?download=1"
    def __init__(
            self,
        ):

        super().__init__()

        
        # TODO: https://github.com/baeklab/PARCtorch/blob/main/tests/conftest.py
        self.dataset_name = ''              # dataset name
        self.grid_type = 'cartesian'        # type of grid. Can only be 'cartesian' at the moment. TODO: 'mesh'
        self.n_spatial_dims = 1             # spatial dimensions
        self.dimensions = {}
        self.boundary_conditions = {}
        self.constants = {}         # physics parameters that are constant across all samples
        self.data = {}              # 

        self.filelist = {}
                

        # Set up data directory
        data_dir = None
        if data_dir is None:
            data_dir = os.path.join(CACHE_DIR, 'datasets', 'NavierStokes')
        self.data_dir = data_dir
        self.zip_dir = os.path.join(self.data_dir, 'NavierStokes.zip')

        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)

        # Download and unzip data if it doesn't exist already.
        self.download(force=False)

        # Read data
        self.load()

        
        # TODO: Lazy loading (Future; low priority)
        pass

    # number of trajectories or simulation instances
    @property
    def n_trajectories(self):
        return len(self.filelist)
    
    def download(self, force=False):
        if not os.path.exists(self.zip_dir):
            download(self.url, self.zip_dir)
            extract_zip(self.zip_dir, self.data_dir)

    def load(self, data_dir=None):
        if data_dir is None:
            data_dir = self.data_dir

        filelist = Path(data_dir).glob('test/*.npy')
        
        
        self.dataset_name = 'Navier Stokes'
        self.grid_type = 'cartesian'
        self.n_spatial_dims = 2
        self.dimensions = {
            'time': Data(torch.linspace(0,1,39)),
            'x': Data(torch.linspace(0,1,256)),
            'y': Data(torch.linspace(0,1,128)),
        }
        # self.constants = {}
        # self.boundary_conditions = {}

        self.filelist = []
        reynolds_numbers = []
        pressure_fields = []
        velocity_fields = []
        for filepath in filelist:
            reynolds_numbers.append( int(filepath.stem.split('_')[-1]) )
            fields = np.load(filepath)
            pressure_fields.append(fields[:,0,:,:])
            velocity_fields.append(fields[:,2:,:,:])
            self.filelist.append(filepath)
    
        self.data = {
            'Reynolds Number': Data(
                torch.tensor(reynolds_numbers),
                traj_varying=True,
                time_varying=False,
                space_varying=[False, False],
            ),
            'Pressure': Data(
                torch.tensor(pressure_fields),
                traj_varying=True,
                time_varying=True,
                space_varying=[True, True],
            ),
            'Velocity': Data(
                torch.tensor(velocity_fields),
                traj_varying=True,
                time_varying=True,
                space_varying=[True, True],
            ),
        }

        

    # def __getitem__(self):
    
    # def __len__(self):

    # def normalize(self):

    # train_ds = ds.filter( torch.reduce_sum(ds.data['morphology'], dim=[1,2]) < 100 )

    def filter(self, mask):
        '''
        train = ds.filter( ds[ds.physics_parameters[0]] < 1000 )
        test = ds.filter( ds.meta.physics_parameters[0] >= 1000 )
        '''
        assert( len(mask.shape) == 1 )    # TODO: error message
        assert( len(mask) == self.n_trajectories ) # TODO: error message

        indices = torch.nonzero(mask)
        ds = Dataset()  # TODO: Copy constructor
        ds.filelist = self.filelist(indices)   # TODO: Deep copy?
        ds.data = self.data[indices]   # TODO: Deep copy?
        ds.grid_type = self.grid_type
        ds.n_spatial_dims = self.n_spatial_dims
        ds.data_dir = self.data_dir
        ds.dimensions = self.dimensions
        ds.boundary_conditions = self.boundary_conditions
        ds.constants = self.constants
        # ds.dataset_name
                
        

ds = Dataset()
print(ds.data['Reynolds Number'])
print(ds.data['Reynolds Number'][0])
print(ds.data['Reynolds Number'] < 1000)
print(ds.data['Reynolds Number'] < torch.tensor([1,2,3,4,1000,1000,1000,1000]))
print(ds.data['Reynolds Number'].rank)
print(ds.data['Reynolds Number'].type)

print(ds.data['Pressure'].rank)
print(ds.data['Pressure'].type)
print(ds.data['Pressure'].shape)
torch.mean(ds.data['Pressure'], dim=[1,2,3])

<PARC Data Class: tensor([ 300, 3000,  400,  550,  700,  800, 8000,  950]), traj_varying=True, time_varying=False, space_varying=[False, False]>
tensor(300)
tensor([ True, False,  True,  True,  True,  True, False,  True])
tensor([False, False, False, False,  True,  True, False,  True])
0
Variable time-independent scalar
0
Variable time-dependent scalar field
torch.Size([8, 39, 128, 256])


tensor([ -64.7932, 1491.8637,  -35.3208,    4.4460,   53.8535,  123.9229,
        2062.4870,  269.5076], dtype=torch.float64)

torch.Size([8, 39, 128, 256])


AttributeError: 'Data' object has no attribute 'value'

In [1]:
from PARCtorch.data.dataset import GenericPhysicsDataset
from PARCtorch.utils.common import CACHE_DIR

from PARCtorch.datasets.utils import download, extract_zip

import os


class NavierStokes(GenericPhysicsDataset):

    """
    Navier-Stokes data set. TODO: More details
    
    Args:
        split: Expected data split. Can be `train`, `test` TODO
        data_dir: Directory to read/write data. Defaults to None, in which case,
                  data will be stored in `CACHE_DIR/datasets/NavierStokes`.
                  If the data does not exist in the specified directory,
                  it will be automatically downloaded.
        future_steps: Number of timesteps in the future the model will predict.
                      Must be between 1 (single step prediction) and TODO (default).
    """

    url = "https://zenodo.org/records/13909869/files/NavierStokes.zip?download=1"

    def __init__(
        self, split=None, data_dir=None, future_steps=2,
    ):
        super().__init__()

        # Set up data directory
        if data_dir is None:
            data_dir = os.path.join(CACHE_DIR, 'datasets', 'NavierStokes')
        self.data_dir = data_dir
        self.zip_dir = os.path.join(self.data_dir, 'NavierStokes.zip')

        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)

        # Download and unzip data if it doesn't exist already.
        self.download(force=False)

        from pathlib import Path
        filelist = Path(self.data_dir).glob('*.*')


    def download(self, force=False):
        if not os.path.exists(self.zip_dir):
            download(self.url, self.zip_dir)
            extract_zip(self.zip_dir, self.data_dir)


In [2]:
data = NavierStokes()
print(data)

TypeError: GenericPhysicsDataset.__init__() missing 1 required positional argument: 'data_dirs'

In [ ]:
import PARCtorch as parc
parc.datasets.NavierStokes()

AttributeError: module 'PARCtorch' has no attribute 'datasets'

## Compute Data Normalization
<p>In this first step we preform min-max normalization - <code>compute_min_max</code> is used to calculate all minimum and maximum values across all provided datasets. This is an important step as normalization improves the predictive accuracy of the model by making sure data falls within a standardized numerical range.</p>

<p>After running this cell the file <strong>ns_min_max.json</strong> will contain parameters to consistently scale data during training and prediction.</p>

In [ ]:
from pathlib import Path
from PARCtorch.data.normalization import compute_min_max

# Define data directories
train_dir = Path("../data/navier_stokes/train")
test_dir = Path("../data/navier_stokes/test")
min_max_file = train_dir.parent / "ns_min_max.json"

compute_min_max([train_dir, test_dir], min_max_file)

## Data Loader for Training
<p> We next create a <strong>DataLoader</strong> for training. The DataLoader in PyTorch is a crucial utility that facilitates efficient data handling for training and evaluating machine learning models. It abstracts the process of fetching, batching, and shuffling data, ensuring that the model is fed with properly formatted inputs in an optimal way. Specifically, it helps with:</p>

<ul>
    <li><strong>Batching:</strong> Splitting large datasets into smaller, manageable batches to avoid memory overload and enable parallel processing.</li>
    <li><strong>Shuffling:</strong> Randomly ordering data to prevent the model from learning patterns related to the sequence of data (particularly important in training to reduce overfitting).</li>
    <li><strong>Parallel Loading:</strong> It allows the data to be loaded asynchronously using multiple workers, speeding up the training process by loading the next batch while the current one is being processed by the model.</li>
    <li><strong>Custom Collation:</strong> The <code>collate_fn</code> allows customization of how batches are combined, which is essential for complex datasets that require specific handling.</li>
</ul>


In [ ]:
# Now import the utilities
import torch
from torch.utils.data import DataLoader

# Now import the utilities
from PARCtorch.data.dataset import (
    GenericPhysicsDataset,
    custom_collate_fn,
    InitialConditionDataset,
    initial_condition_collate_fn,
)
from PARCtorch.utilities.viz import (
    visualize_channels,
    save_gifs_with_ground_truth,
)
future_steps = 1
batch_size = 8

# Initialize the dataset
train_dataset = GenericPhysicsDataset(
    data_dirs=[train_dir],
    future_steps=future_steps,
    min_max_path=min_max_file,
)

# Create DataLoader for training dataset
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
    collate_fn=custom_collate_fn,
)

## Visualize the Data - Check it was Loaded Properly
<p> We are now going to visualize our data to verify if the correct data is loading and formatting. In the cell below we are iterating over the previously defined DataLoader and fetching one batch of data. The batch is then split up into the following variables:</p>

<ul>
    <li><code>ic</code>: Inital conditions - the initial state of the physical system</li>
    <li><code>t0</code>: Starting time-step </li>
    <li><code>t1</code>: Ending time-step </li>
    <li><code>target</code>: Ground truth future states</li>
</ul>




In [ ]:
# Fetch a batch and visualize
for batch in train_loader:
    ic, t0, t1, target = batch
    channel_names = [
        "Pressure (P)",
        "Reynolds (R)",
        "Velocity U",
        "Velocity V",
    ]
    custom_cmaps = ["seismic", "seismic", "seismic", "seismic"]

    visualize_channels(
        ic,
        t0,
        t1,
        target,
        channel_names=channel_names,
        channel_cmaps=custom_cmaps,
    )
    break  # Visualize one batch for now

<h2>Build Your PARC Model</h2>
<p>In this section, we are constructing a <strong>PARCv2 model</strong>, which is designed to handle spatiotemporal data, such as fluid dynamics simulations. The model leverages various components, including <em>differentiators</em> and <em>integrators</em>, to solve physical equations like Navier-Stokes.</p>

<p><strong>Key Components:</strong></p>
<ul>
    <li><strong>UNet:</strong> The UNet architecture is used for feature extraction, transforming the input data into a higher-dimensional representation. This helps the model capture complex patterns in the physical simulation data.</li>
    <li><strong>FiniteDifference:</strong> This differentiator approximates the gradients (or derivatives) of the input data using a finite difference method, which is important for calculating advection and diffusion processes in fluid dynamics.</li>
    <li><strong>Heun Integrator:</strong> Heun’s method is an improved version of Euler's method, used here to integrate the equations of motion more accurately over time.</li>
    <li><strong>Differentiator:</strong> This module calculates the advection and diffusion terms based on specific channel indices, such as velocity in the x and y directions (u and v).</li>
    <li><strong>Integrator:</strong> The integrator applies Heun’s method to combine the differentiated terms and solve the Poisson equation, ensuring the physical constraints of the system are respected.</li>
</ul>

<p>The model is then wrapped into the <strong>PARCv2</strong> class, which combines the differentiator, integrator, and loss function (<code>L1Loss</code>). Finally, an <code>Adam</code> optimizer is initialized to train the model by adjusting its parameters to minimize the error between predictions and ground truth data.</p>

<p>This setup allows the model to learn how to predict future states in complex physical systems by embedding domain-specific knowledge.</p>


In [5]:
import sys

sys.path.append("../../")

In [6]:
from PARCtorch.PARCv2 import PARCv2
from PARCtorch.differentiator.differentiator import ADRDifferentiator
from PARCtorch.differentiator.finitedifference import FiniteDifference
from PARCtorch.integrator.integrator import Integrator
from PARCtorch.integrator.heun import Heun
from PARCtorch.utilities.unet import UNet

from torch.optim import Adam

In [7]:
# Defining Navier-Stokes variables:
# p = pressure, re = Reynolds number, u = velocity in x-direction, v = velocity in y-direction
# Advection (Adv) and Diffusion (Dif) will be calculated on u, v
# Poisson equation (Poi) will be calculated on pressure (p)

n_fe_features = 128  # Number of features extracted by the UNet

# Initialize the UNet architecture for feature extraction
unet_ns = UNet(
    [64, 64 * 2, 64 * 4, 64 * 8, 64 * 16],  
    4,  
    n_fe_features,  
    up_block_use_concat=[False, True, False, True],  
    skip_connection_indices=[2, 0],  
)

# Initialize finite difference method for numerical differentiation
right_diff = FiniteDifference(padding_mode="replicate").cuda()  # Use replication padding to handle boundary conditions

# Initialize Heun's method for numerical integration
heun_int = Heun().cuda() 

# Create the Differentiator, responsible for calculating advection and diffusion
diff_ns = ADRDifferentiator(
    2,  
    n_fe_features,  
    [2, 3],  
    [2, 3],  
    unet_ns,  
    "constant",  
    right_diff,  
    False 
).cuda()

# Create the Integrator, responsible for solving the Poisson equation and performing integration
ns_int = Integrator(
    True,  
    [(0, 2, 3, 1)],  
    heun_int,  
    [None, None, None, None],  
    "constant",  
    right_diff,  
).cuda()

# Define the loss function (L1 Loss is typically used for regression tasks)
criterion = torch.nn.L1Loss().cuda()

# Initialize the PARCv2 model with the differentiator, integrator, and loss function
model = PARCv2(diff_ns, ns_int, criterion).cuda()

# Set up the optimizer (Adam is a popular choice for adaptive learning rate optimization)
optimizer = Adam(model.parameters(), lr=1e-5)  


## Train the Model 
<p>Now, we will be training the PARC model using the training data and the paramaters we have setup above. Here we iterate over the dataset given by <code>train_loader</code> batch-by-batch and in each iteration we: </p>

<ul>
    <li>Compute model predictions</li>
    <li>Calculate error loss between predictions and ground truth values via the <code>criterion</code> loss function </li>
    <li>Update the model parameters based on error via the <code>optimizer</code></li>
</ul>



In [ ]:
from PARCtorch.train import train_model

# Example usage:
train_model(
    model,
    train_loader,
    criterion,
    optimizer,
    num_epochs=1,
    save_dir=train_dir.parent,
    app="ns",
)


## Load the Model
<p> Post training, we can now load the previously trained model weights into our PARC model for evalutation. Here, we locate the model weights and load them into the exisiting PARC model architecture in order to prep for visualization.</p>

In [ ]:
from PARCtorch.utilities.load import load_model_weights

# Example Usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights_path = (
    train_dir.parent / "model.pth"  # Replace with your path
)
model = load_model_weights(model, model_weights_path, device)

## Sequence DataLoader
<p>We now create a sequence DataLoader designed to test the dataset - providing the initial conditions as <code>t=0</code> for generating predictiond over multiple timesteps. Using <code>DataLoader</code> we wrap the dataset for efficent batching and processing.</p>

<p>We also create the Initial Condition Dataset, which loades the initial state for the physical system, the previously computed normalization parameters, and prepares the dataset to be fed into the model. We also 

In [ ]:
# Initialize the dataset
future_steps = 10
seq_dataset = InitialConditionDataset(
    data_dirs=[test_dir],
    future_steps=future_steps,
    min_max_path=min_max_file,
)

# Create DataLoader for training dataset
seq_loader = DataLoader(
    seq_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
    collate_fn=initial_condition_collate_fn,
)

## Ground Truth Loader
<p>Here, we set up the Ground Truth Loader to provide a comparison of the actual future states of the system against the models predictions. Similar set up to previous <code>DataLoader</code> setups in this notebook.</p>

In [ ]:
# Initialize the dataset
gt_dataset = GenericPhysicsDataset(
    data_dirs=[test_dir],
    future_steps=future_steps,
    min_max_path=min_max_file,
)

# Create DataLoader for training dataset
gt_loader = DataLoader(
    gt_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
    collate_fn=custom_collate_fn,
)

## Visualize the Results
<p> We now finially visualize our results. Here, we run inferences using thr trained PARC model's predictions against the ground truth. Visualization gifs of the predictions and ground truth are outputted - just runs on the first batch for demo.</p>

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define channel names and colormaps
channels = ["pressure", "Reynolds", "u", "v"]  # Adjust as per your data
cmaps = [
    "viridis",
    "plasma",
    "inferno",
    "magma",
]  # Adjust as per your preference

# Iterate through both DataLoaders simultaneously
for seq_batch, test_batch in zip(seq_loader, gt_loader):
    # Extract data from initial condition loader
    ic, t0, t1, _ = (
        seq_batch  # Shape: [batch_size, channels, height, width], scalar, tensor, _
    )

    # Extract data from ground truth loader
    gt_ic, gt_t0, gt_t1, ground_truth = (
        test_batch  # ground_truth shape: [timesteps, batch_size, channels, height, width]
    )

    # Move data to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ic = ic.to(device)
    t0 = t0.to(device)
    t1 = t1.to(device)
    ground_truth = ground_truth.to(device)

    # Make predictions using the model
    with torch.no_grad():
        predictions = model(
            ic, t0, t1
        )  # Shape: [future_steps, batch_size, channels, height, width]

    print("Predictions shape:", predictions.shape)
    print(
        "Sample prediction for timestep 1:", predictions[:, 0, :, :, :].shape
    )

    # If you want to visualize more samples in the batch, loop through batch indices
    # For example, to visualize all samples in the batch:
    for batch_idx in range(ic.size(0)):
        save_gifs_with_ground_truth(
            predictions=predictions,
            ground_truth=ground_truth,
            channels=channels,
            cmaps=cmaps,
            filename_prefix=f"comparison_batch{batch_idx}",
            interval=0.2,
            batch_idx=batch_idx,
        )
        break

    break  # Remove this if you want to process the entire dataset